# Setting Everything Up

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
#import the file from the web
import requests
from bs4 import BeautifulSoup
import pandas as pd
from unicodedata import normalize
import re
from datetime import datetime
from pandas import read_csv 
from sklearn.feature_extraction.text import CountVectorizer
import json
from flatten_json import flatten


# I can give a number or use None to remove maximum ceiling & display all columns
pd.options.display.max_columns = None

# # I want to be able to see the entire narrative, so remove the maximum width for each column
# pd.options.display.max_colwidth = None

# pd.options.display.float_format = '{:,.0f}'.format

import string

%matplotlib inline

In [3]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (8, 8)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [4]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [5]:
%%R

# My commonly used R imports

require('tidyverse')

R[write to console]: Loading required package: tidyverse



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [6]:
#To Download PDFs
from urllib.parse import urljoin

In [7]:
#To import camelot and PDF-related items
import camelot
import ghostscript
import sys

In [8]:
%%R

library(jpeg)
library(wordcloud)
library(RColorBrewer)
library(wordcloud2)
library(tm)


R[write to console]: Loading required package: RColorBrewer

R[write to console]: Loading required package: NLP

R[write to console]: 
Attaching package: ‘NLP’


R[write to console]: The following object is masked from ‘package:ggplot2’:

    annotate




In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
%%R
library(vistime)
library(shiny)
library(plotly)
#for timeline plotting

R[write to console]: 
Attaching package: ‘plotly’


R[write to console]: The following object is masked from ‘package:ggplot2’:

    last_plot


R[write to console]: The following object is masked from ‘package:stats’:

    filter


R[write to console]: The following object is masked from ‘package:graphics’:

    layout




In [11]:
from collections import Counter


In [12]:
# for playwright stuff
import time
from playwright.async_api import async_playwright
import asyncio
import nest_asyncio


In [13]:
# !pip install flask


 # Getting 10K

# Putting together code for finding the 10k link

#### Reading the data

In [14]:
# # whatever title is input
# one_title_searchable="Apple Inc"

In [17]:
# this is final


async def extracting_stock_compensation_for_one_company():
    

    async def getting_10_k_link(one_title_searchable):
        #### ON THE SEC HOME PAGE
        # going to sec page
        sec_home_playwright = await async_playwright().start()
        sec_home_browser = await sec_home_playwright.chromium.launch(headless = False)
        sec_home_page = await sec_home_browser.new_page()

        await sec_home_page.goto("https://www.sec.gov/edgar/searchedgar/companysearch")
        time.sleep(6)

        try:

            # input company name (here is where we can enter title variable from df from csv)

            await sec_home_page.fill("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/input", one_title_searchable)
            time.sleep(6)

            # click search

            await sec_home_page.click("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/button")
            time.sleep(6)

            #============
            # ON THE NEW PAGE RESULTING WITH ALL COMPANY DOCS
            #input 10K for search
            await sec_home_page.fill("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[1]/input", "10-K")
            time.sleep(6)

            # click search

            await sec_home_page.click("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[5]/input[1]")
            time.sleep(6)


            #get all 10K entries
            list_of_all_ten_k_entries_response= await sec_home_page.content()

            list_of_all_ten_k_entries_doc = BeautifulSoup(list_of_all_ten_k_entries_response, 'html.parser')

            #get the first 10K
            list_of_all_ten_k_entries_table = list_of_all_ten_k_entries_doc.find('table', class_='tableFile2')

            ten_k_entries_rows = list_of_all_ten_k_entries_table.find_all('tr')  # get the first row because it has the most recent 10K

            most_recent_ten_k_link = None

            for one_ten_k_entries_row in ten_k_entries_rows:
                one_ten_k_entries_row_cells = one_ten_k_entries_row.find_all('td')  # Assuming each cell is represented by a <td> tag
            #     print(one_ten_k_entries_row_cells)

                for one_ten_k_entries_row_cell in one_ten_k_entries_row_cells:
            #         print(one_ten_k_entries_row_cell)
                    if one_ten_k_entries_row_cell.text.strip() == '10-K':
                        ten_k_document_link_href = one_ten_k_entries_row_cell.find_next('td').find('a')['href']

                        ten_k_document_link_base = "https://www.sec.gov/"
                        ten_k_document_link_tail = ten_k_document_link_href
                        ten_k_document_link = ten_k_document_link_base + ten_k_document_link_tail

            #             print(document_link)

                        # Store the first 10-K link and break out of the loop
                        most_recent_ten_k_link = ten_k_document_link
                        break

                if most_recent_ten_k_link is not None:
                    break


            # ONCE I HAVE THE TEN K DOC LINK 
            # 3. Now on the page with the document and graphics table

            ten_k_doc_link_playwright = await async_playwright().start()
            ten_k_doc_link_browser = await ten_k_doc_link_playwright.chromium.launch(headless = False)
            ten_k_doc_link_page = await ten_k_doc_link_browser.new_page()

            time.sleep(6)

            await ten_k_doc_link_page.goto(most_recent_ten_k_link)
            time.sleep(6)



            # Initiate the html and beautiful soup content 

            list_of_doc_and_graphics_response= await ten_k_doc_link_page.content()

            list_of_doc_and_graphics_soup_doc = BeautifulSoup(list_of_doc_and_graphics_response, 'html.parser')

            # get actual_full_ten_k_document_link
            list_of_doc_and_graphics_table = list_of_doc_and_graphics_soup_doc.find('table', class_='tableFile')


            list_of_doc_and_graphics_rows = list_of_doc_and_graphics_table.find_all('tr')  # Assuming each row is represented by a <tr> tag

            for list_of_doc_and_graphics_row in list_of_doc_and_graphics_rows:
                list_of_doc_and_graphics_cells = list_of_doc_and_graphics_row.find_all('td')  # Assuming each cell is represented by a <td> tag
            #     print(cells)
                for list_of_doc_and_graphics_cell in list_of_doc_and_graphics_cells:
                    if list_of_doc_and_graphics_cell.text.strip() == '10-K':
                        actual_document_link = list_of_doc_and_graphics_cell.find_next('td').find('a')
                        if actual_document_link is not None:
                            actual_ten_k_document_href = actual_document_link['href']
            #                 print("Documents href:", document_href)
                            actual_ten_k_document_link_base="https://www.sec.gov/"
                            actual_ten_k_document_link_tail=actual_ten_k_document_href
                            actual_full_ten_k_document_link=actual_ten_k_document_link_base+actual_ten_k_document_link_tail
#                             global x
                            x=actual_full_ten_k_document_link
                            print(actual_full_ten_k_document_link)
                            print("#########")

                            


        except:
            pass

        finally:
            # Close the page and browser
            await ten_k_doc_link_page.close()
            await ten_k_doc_link_browser.close()
            await ten_k_doc_link_playwright.stop()

            # Close the sec_home page and browser
            await sec_home_page.close()
            await sec_home_browser.close()
            await sec_home_playwright.stop()
            return actual_full_ten_k_document_link
    
    
    
    x = await getting_10_k_link(one_title_searchable)
    
    
    
    
    
    
    # with change for link


    list_of_actual_full_ten_k_document_links = ["https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000156459022026876/msft-10k_20220630.htm"]

    actual_list_of_actual_full_ten_k_document_links = ["https://www.sec.gov//ix?doc=/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm","https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000156459022026876/msft-10k_20220630.htm","https://www.sec.gov/ix?doc=/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm"]

    async def get_actual_full_ten_k_document_content(x):
        playwright = await async_playwright().start()
        browser = await playwright.chromium.launch(headless=False)
    #     browser = await playwright.firefox.launch(headless=False)

        page = await browser.new_page()

        await page.goto(x)
        time.sleep(10)

        while True:
            # Get the number of spans on the page
            spans_assigned = await page.query_selector_all("span")
            test = len(spans_assigned)

            # Scroll in increments of 5000 pixels
            await page.evaluate('''() => {
                let distance = 5000;
                window.scrollBy(0, distance);
            }''')
    #         await asyncio.sleep(3)

            time.sleep(10)


            spans_assigned_two = await page.query_selector_all("span")
            test2 = len(spans_assigned_two)
            if test == test2:
                break

        # Get the content of the page
        actual_full_ten_k_document_content = await page.content()
        time.sleep(10)

        await browser.close()
        time.sleep(10)
        await page.close()  # Add this line to close the page
        time.sleep(10)

        return actual_full_ten_k_document_content


    async def process_actual_full_ten_k_document(actual_full_ten_k_document_content):
        actual_full_ten_k_document_soup = BeautifulSoup(actual_full_ten_k_document_content, 'html.parser')
        return actual_full_ten_k_document_soup


    async def main():
        dict_titles=['actual_full_ten_k_document_link_collection',
                     'table_length',
                     'list_of_stock_based_values',
                     'first_stock_based_compensation',
                     'second_stock_based_compensation',
                     'third_stock_based_compensation']

        possible_titles = ['CONSOLIDATED STATEMENTS OF CASH FLOWS',
                           'CASH FLOWS STATEMENTS',
                           'Consolidated Statements of Cash Flows',
                           'Consolidated Statement of Cash Flows',
                          'CONSOLIDATED STATEMENT OF CASH FLOWS',
                          'STATEMENTS OF CONSOLIDATED CASH FLOWS',
                          'Consolidated Cash Flow Statement',
                          'Consolidated Statement of Cash Flow for the Years Ended December 31',
                          'STATEMENT OF CASH FLOWS',
                          'CONSOLIDATED STATEMENTS OF CASH FLOW',
                          'Statements of Consolidated Cash Flows']

        stock_related_texts = [
            'Stock-based compensation expense',
            'Stock-based compensation expense',
            'Share-based compensation expense',
            'Stock-based compensation',
            'Share-based compensation'
        ]

        all_ten_k_document_lists = []


    #     list_of_example_actual_full_ten_k_document_soup_doc = []

        # Define the group size
        group_size = 2
        items=[x]

        # Iterate over groups of items
        for start_index in range(0, len(items), group_size):
            group_items = items[start_index:start_index + group_size]

            list_of_example_actual_full_ten_k_document_soup_doc = []


    #         # Iterate over items within the group
    #         for item in group_items:
    #             print(item)

    #         # Add a separator for each group
    #         print("Group separator")




            url_to_parsed_content = {}


            for actual_full_ten_k_document_link in group_items:
                try:
                    actual_full_ten_k_document_link_collection = actual_full_ten_k_document_link
                    print(actual_full_ten_k_document_link)

                    for _ in range(3):
                        try:
                            actual_full_ten_k_document_content = await get_actual_full_ten_k_document_content(actual_full_ten_k_document_link)
                            example_actual_full_ten_k_document_soup_doc = await process_actual_full_ten_k_document(actual_full_ten_k_document_content)
                            list_of_example_actual_full_ten_k_document_soup_doc.append(example_actual_full_ten_k_document_soup_doc)
                                # Store the parsed content in the dictionary.
                            url_to_parsed_content[actual_full_ten_k_document_link] = example_actual_full_ten_k_document_soup_doc

                            break  # Break the loop if successful
                        except Exception as e:
                            print(f"An error occurred: {str(e)}. Retrying...")
                            time.sleep(15)
                except:
                    pass

            for actual_full_ten_k_document_link, one_actual_full_ten_k_document_soup_doc in url_to_parsed_content.items():
                ten_k_detail_list=[]

                if one_actual_full_ten_k_document_soup_doc:

                    elements = one_actual_full_ten_k_document_soup_doc.find_all(text=possible_titles)

                    if not elements:
                        print("Titles not found in the HTML.")
                        continue

                    for element in elements:
                        if element.strip() in possible_titles:
                            following_elements = element.find_all_next()
                            table = None

                            # Iterate through following elements and stop when a table is found or after three components
                            component_count = 0
                            for next_element in following_elements:
                                if next_element.name == 'table' and any(text in next_element.get_text() for text in stock_related_texts):
                                    table = next_element
                                    break
                                elif next_element.name in ['p', 'span']:
                                    component_count += 1
                                    if component_count >= 5:
                                        break

                            if table is None:
                                tables = one_actual_full_ten_k_document_soup_doc.find_all("table")
                                for t in tables:     
                                    if t:
                                        table_text = t.get_text()
                                        if any(title in table_text for title in possible_titles) and any(stock_related_text in table_text for stock_related_text in stock_related_texts):
                                            rows = t.find_all('tr')
                                            table_length = len(rows)
                                            print(f"Table Length from within: {len(rows)}") 
                                        break

                            if table:
                                rows = table.find_all('tr')
                                table_length=len(rows)

                                print(f"Table Length: {len(rows)}")                        

                                break



                    else:
                        print("No table found within or after the possible titles in the HTML.")

                else:
                    print("Document not processed, skipping")


                if table:
                    target_td = table.find('td', text=stock_related_texts)
                else:
                    target_td = None

                if target_td:
                    # Move up to the <tr> containing the <td> with "share-based compensation"
                    target_tr = target_td.find_parent('tr')

                    # Extract all the <td> values from the <tr> and put them in a column
                    values_in_column = [td.get_text(strip=True) for td in target_tr.find_all('td')]

                    # Output the values in the column
                    if values_in_column:
                        stock_stock_based_compensation_title=values_in_column[0]
                        list_of_stock_based_values=values_in_column
                        numbers_only_values_in_column = [entry for entry in values_in_column if any(char.isdigit() for char in entry)]
                        first_stock_based_compensation=numbers_only_values_in_column[0]
                        second_stock_based_compensation=numbers_only_values_in_column[1]
                        third_stock_based_compensation=numbers_only_values_in_column[2]
                        for value in values_in_column:
                            print(value)

                    ten_k_detail_list.append(actual_full_ten_k_document_link)
                    ten_k_detail_list.append(table_length)
                    ten_k_detail_list.append(list_of_stock_based_values)
                    ten_k_detail_list.append(first_stock_based_compensation)
                    ten_k_detail_list.append(second_stock_based_compensation)
                    ten_k_detail_list.append(third_stock_based_compensation)

                    first_rep = dict(zip(dict_titles, ten_k_detail_list))
                    all_ten_k_document_lists.append(first_rep)



                else:
                    print("The 'Share-Based Compensation' row not found!")
                time.sleep(10)

    #     print(all_ten_k_document_lists)
        return all_ten_k_document_lists


    # Run the main coroutine
    nest_asyncio.apply()

    # Use asyncio.run() to run the main coroutine
    all_ten_k_document_lists_of_dicts=asyncio.run(main())
    
    

In [19]:
!brew tap heroku/brew && brew install heroku


Running `brew update --auto-update`...
==> Homebrew collects anonymous analytics.
Read the analytics documentation (and how to opt-out) here:
  https://docs.brew.sh/Analytics
No analytics have been recorded yet (nor will be during this `brew` run).

==> homebrew/core is old and unneeded, untapping to save space...
Untapping homebrew/core...
Untapped 3 commands and 6530 formulae (7,125 files, 672.5MB).
==> homebrew/cask is old and unneeded, untapping to save space...
Untapping homebrew/cask...
Untapped 4161 casks (4,328 files, 362.6MB).
==> Downloading https://formulae.brew.sh/api/formula_tap_migrations.jws.json
######################################################################### 100.0%
==> Auto-updated Homebrew!
Updated 4 taps (homebrew/services, caskroom/cask, homebrew/core and homebrew/cask).
==> New Formulae
adb-enhanced                             medusa
aerleon                                  meson-python
alass                                    mgis
apify-cli              

######################################################################### 100.0%
==> Installing heroku from heroku/brew
Update them from Software Update in System Preferences.

If that doesn't show you any updates, run:
  sudo rm -rf /Library/Developer/CommandLineTools
  sudo xcode-select --install

Alternatively, manually download them from:
  https://developer.apple.com/download/all/.
You should download the Command Line Tools for Xcode 14.2.

==> Caveats
To use the Heroku CLI's autocomplete --
  Via homebrew's shell completion:
    1) Follow homebrew's install instructions https://docs.brew.sh/Shell-Completion
        NOTE: For zsh, as the instructions mention, be sure compinit is autoloaded
              and called, either explicitly or via a framework like oh-my-zsh.
    2) Then run
      $ heroku autocomplete --refresh-cache
  OR
  Use our standalone setup:
    1) Run and follow the install steps:
      $ heroku autocomplete

zsh completions have been installed to:
  /opt/homebre

In [18]:
await extracting_stock_compensation_for_one_company()

UnboundLocalError: cannot access local variable 'ten_k_doc_link_page' where it is not associated with a value

In [ ]:
# whatever title is input
one_title_searchable="Apple Inc"

In [ ]:
# this is final

async def getting_10_k_link(one_title_searchable):
    #### ON THE SEC HOME PAGE
    # going to sec page
    sec_home_playwright = await async_playwright().start()
    sec_home_browser = await sec_home_playwright.chromium.launch(headless = False)
    sec_home_page = await sec_home_browser.new_page()

    await sec_home_page.goto("https://www.sec.gov/edgar/searchedgar/companysearch")
    time.sleep(6)

    try:

        # input company name (here is where we can enter title variable from df from csv)

        await sec_home_page.fill("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/input", one_title_searchable)
        time.sleep(6)

        # click search

        await sec_home_page.click("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/button")
        time.sleep(6)

        #============
        # ON THE NEW PAGE RESULTING WITH ALL COMPANY DOCS
        #input 10K for search
        await sec_home_page.fill("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[1]/input", "10-K")
        time.sleep(6)

        # click search

        await sec_home_page.click("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[5]/input[1]")
        time.sleep(6)


        #get all 10K entries
        list_of_all_ten_k_entries_response= await sec_home_page.content()

        list_of_all_ten_k_entries_doc = BeautifulSoup(list_of_all_ten_k_entries_response, 'html.parser')

        #get the first 10K
        list_of_all_ten_k_entries_table = list_of_all_ten_k_entries_doc.find('table', class_='tableFile2')

        ten_k_entries_rows = list_of_all_ten_k_entries_table.find_all('tr')  # get the first row because it has the most recent 10K

        most_recent_ten_k_link = None

        for one_ten_k_entries_row in ten_k_entries_rows:
            one_ten_k_entries_row_cells = one_ten_k_entries_row.find_all('td')  # Assuming each cell is represented by a <td> tag
        #     print(one_ten_k_entries_row_cells)

            for one_ten_k_entries_row_cell in one_ten_k_entries_row_cells:
        #         print(one_ten_k_entries_row_cell)
                if one_ten_k_entries_row_cell.text.strip() == '10-K':
                    ten_k_document_link_href = one_ten_k_entries_row_cell.find_next('td').find('a')['href']

                    ten_k_document_link_base = "https://www.sec.gov/"
                    ten_k_document_link_tail = ten_k_document_link_href
                    ten_k_document_link = ten_k_document_link_base + ten_k_document_link_tail

        #             print(document_link)

                    # Store the first 10-K link and break out of the loop
                    most_recent_ten_k_link = ten_k_document_link
                    break

            if most_recent_ten_k_link is not None:
                break


        # ONCE I HAVE THE TEN K DOC LINK 
        # 3. Now on the page with the document and graphics table

        ten_k_doc_link_playwright = await async_playwright().start()
        ten_k_doc_link_browser = await ten_k_doc_link_playwright.chromium.launch(headless = False)
        ten_k_doc_link_page = await ten_k_doc_link_browser.new_page()

        time.sleep(6)

        await ten_k_doc_link_page.goto(most_recent_ten_k_link)
        time.sleep(6)



        # Initiate the html and beautiful soup content 

        list_of_doc_and_graphics_response= await ten_k_doc_link_page.content()

        list_of_doc_and_graphics_soup_doc = BeautifulSoup(list_of_doc_and_graphics_response, 'html.parser')

        # get actual_full_ten_k_document_link
        list_of_doc_and_graphics_table = list_of_doc_and_graphics_soup_doc.find('table', class_='tableFile')


        list_of_doc_and_graphics_rows = list_of_doc_and_graphics_table.find_all('tr')  # Assuming each row is represented by a <tr> tag

        for list_of_doc_and_graphics_row in list_of_doc_and_graphics_rows:
            list_of_doc_and_graphics_cells = list_of_doc_and_graphics_row.find_all('td')  # Assuming each cell is represented by a <td> tag
        #     print(cells)
            for list_of_doc_and_graphics_cell in list_of_doc_and_graphics_cells:
                if list_of_doc_and_graphics_cell.text.strip() == '10-K':
                    actual_document_link = list_of_doc_and_graphics_cell.find_next('td').find('a')
                    if actual_document_link is not None:
                        actual_ten_k_document_href = actual_document_link['href']
        #                 print("Documents href:", document_href)
                        actual_ten_k_document_link_base="https://www.sec.gov/"
                        actual_ten_k_document_link_tail=actual_ten_k_document_href
                        actual_full_ten_k_document_link=actual_ten_k_document_link_base+actual_ten_k_document_link_tail
                        actual_full_ten_k_document_link=x
                        print(actual_full_ten_k_document_link)
                        print("#########")

                        # Add the values to the DataFrame
                        df_stock_top_us_plus_sec_titles.loc[df_stock_top_us_plus_sec_titles['title_searchable'] == one_title_searchable, 'ten_k_document_link'] = ten_k_document_link
                        df_stock_top_us_plus_sec_titles.loc[df_stock_top_us_plus_sec_titles['title_searchable'] == one_title_searchable, 'actual_full_ten_k_document_link'] = actual_full_ten_k_document_link
    except:
        pass

    finally:
        # Close the page and browser
        await ten_k_doc_link_page.close()
        await ten_k_doc_link_browser.close()
        await ten_k_doc_link_playwright.stop()

        # Close the sec_home page and browser
        await sec_home_page.close()
        await sec_home_browser.close()
        await sec_home_playwright.stop()

In [ ]:
actual_full_ten_k_document_link

# Going into link and getting stock-based compensation

In [ ]:
# this is final
async def extracting_stock_compensation_for_one_company(one_title_searchable):

    async def getting_10_k_link(one_title_searchable):
        #### ON THE SEC HOME PAGE
        # going to sec page
        sec_home_playwright = await async_playwright().start()
        sec_home_browser = await sec_home_playwright.chromium.launch(headless = False)
        sec_home_page = await sec_home_browser.new_page()

        await sec_home_page.goto("https://www.sec.gov/edgar/searchedgar/companysearch")
        time.sleep(6)

        try:

            # input company name (here is where we can enter title variable from df from csv)

            await sec_home_page.fill("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/input", one_title_searchable)
            time.sleep(6)

            # click search

            await sec_home_page.click("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/button")
            time.sleep(6)

            #============
            # ON THE NEW PAGE RESULTING WITH ALL COMPANY DOCS
            #input 10K for search
            await sec_home_page.fill("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[1]/input", "10-K")
            time.sleep(6)

            # click search

            await sec_home_page.click("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[5]/input[1]")
            time.sleep(6)


            #get all 10K entries
            list_of_all_ten_k_entries_response= await sec_home_page.content()

            list_of_all_ten_k_entries_doc = BeautifulSoup(list_of_all_ten_k_entries_response, 'html.parser')

            #get the first 10K
            list_of_all_ten_k_entries_table = list_of_all_ten_k_entries_doc.find('table', class_='tableFile2')

            ten_k_entries_rows = list_of_all_ten_k_entries_table.find_all('tr')  # get the first row because it has the most recent 10K

            most_recent_ten_k_link = None

            for one_ten_k_entries_row in ten_k_entries_rows:
                one_ten_k_entries_row_cells = one_ten_k_entries_row.find_all('td')  # Assuming each cell is represented by a <td> tag
            #     print(one_ten_k_entries_row_cells)

                for one_ten_k_entries_row_cell in one_ten_k_entries_row_cells:
            #         print(one_ten_k_entries_row_cell)
                    if one_ten_k_entries_row_cell.text.strip() == '10-K':
                        ten_k_document_link_href = one_ten_k_entries_row_cell.find_next('td').find('a')['href']

                        ten_k_document_link_base = "https://www.sec.gov/"
                        ten_k_document_link_tail = ten_k_document_link_href
                        ten_k_document_link = ten_k_document_link_base + ten_k_document_link_tail

            #             print(document_link)

                        # Store the first 10-K link and break out of the loop
                        most_recent_ten_k_link = ten_k_document_link
                        break

                if most_recent_ten_k_link is not None:
                    break


            # ONCE I HAVE THE TEN K DOC LINK 
            # 3. Now on the page with the document and graphics table

            ten_k_doc_link_playwright = await async_playwright().start()
            ten_k_doc_link_browser = await ten_k_doc_link_playwright.chromium.launch(headless = False)
            ten_k_doc_link_page = await ten_k_doc_link_browser.new_page()

            time.sleep(6)

            await ten_k_doc_link_page.goto(most_recent_ten_k_link)
            time.sleep(6)



            # Initiate the html and beautiful soup content 

            list_of_doc_and_graphics_response= await ten_k_doc_link_page.content()

            list_of_doc_and_graphics_soup_doc = BeautifulSoup(list_of_doc_and_graphics_response, 'html.parser')

            # get actual_full_ten_k_document_link
            list_of_doc_and_graphics_table = list_of_doc_and_graphics_soup_doc.find('table', class_='tableFile')


            list_of_doc_and_graphics_rows = list_of_doc_and_graphics_table.find_all('tr')  # Assuming each row is represented by a <tr> tag

            for list_of_doc_and_graphics_row in list_of_doc_and_graphics_rows:
                list_of_doc_and_graphics_cells = list_of_doc_and_graphics_row.find_all('td')  # Assuming each cell is represented by a <td> tag
            #     print(cells)
                for list_of_doc_and_graphics_cell in list_of_doc_and_graphics_cells:
                    if list_of_doc_and_graphics_cell.text.strip() == '10-K':
                        actual_document_link = list_of_doc_and_graphics_cell.find_next('td').find('a')
                        if actual_document_link is not None:
                            actual_ten_k_document_href = actual_document_link['href']
            #                 print("Documents href:", document_href)
                            actual_ten_k_document_link_base="https://www.sec.gov/"
                            actual_ten_k_document_link_tail=actual_ten_k_document_href
                            actual_full_ten_k_document_link=actual_ten_k_document_link_base+actual_ten_k_document_link_tail
#                             global x
                            x=actual_full_ten_k_document_link
                            print(actual_full_ten_k_document_link)
                            print("#########")

                            


        except:
            pass

        finally:
            # Close the page and browser
            await ten_k_doc_link_page.close()
            await ten_k_doc_link_browser.close()
            await ten_k_doc_link_playwright.stop()

            # Close the sec_home page and browser
            await sec_home_page.close()
            await sec_home_browser.close()
            await sec_home_playwright.stop()
            return actual_full_ten_k_document_link
    
    
    
    x=one_title_searchable
    
    
    
    
    
    
    # with change for link


    list_of_actual_full_ten_k_document_links = ["https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000156459022026876/msft-10k_20220630.htm"]

    actual_list_of_actual_full_ten_k_document_links = ["https://www.sec.gov//ix?doc=/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm","https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000156459022026876/msft-10k_20220630.htm","https://www.sec.gov/ix?doc=/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm"]

    async def get_actual_full_ten_k_document_content(x):
        playwright = await async_playwright().start()
        browser = await playwright.chromium.launch(headless=False)
    #     browser = await playwright.firefox.launch(headless=False)

        page = await browser.new_page()

        await page.goto(x)
        time.sleep(10)

        while True:
            # Get the number of spans on the page
            spans_assigned = await page.query_selector_all("span")
            test = len(spans_assigned)

            # Scroll in increments of 5000 pixels
            await page.evaluate('''() => {
                let distance = 5000;
                window.scrollBy(0, distance);
            }''')
    #         await asyncio.sleep(3)

            time.sleep(10)


            spans_assigned_two = await page.query_selector_all("span")
            test2 = len(spans_assigned_two)
            if test == test2:
                break

        # Get the content of the page
        actual_full_ten_k_document_content = await page.content()
        time.sleep(10)

        await browser.close()
        time.sleep(10)
        await page.close()  # Add this line to close the page
        time.sleep(10)

        return actual_full_ten_k_document_content


    async def process_actual_full_ten_k_document(actual_full_ten_k_document_content):
        actual_full_ten_k_document_soup = BeautifulSoup(actual_full_ten_k_document_content, 'html.parser')
        return actual_full_ten_k_document_soup


    async def main():
        dict_titles=['actual_full_ten_k_document_link_collection',
                     'table_length',
                     'list_of_stock_based_values',
                     'first_stock_based_compensation',
                     'second_stock_based_compensation',
                     'third_stock_based_compensation']

        possible_titles = ['CONSOLIDATED STATEMENTS OF CASH FLOWS',
                           'CASH FLOWS STATEMENTS',
                           'Consolidated Statements of Cash Flows',
                           'Consolidated Statement of Cash Flows',
                          'CONSOLIDATED STATEMENT OF CASH FLOWS',
                          'STATEMENTS OF CONSOLIDATED CASH FLOWS',
                          'Consolidated Cash Flow Statement',
                          'Consolidated Statement of Cash Flow for the Years Ended December 31',
                          'STATEMENT OF CASH FLOWS',
                          'CONSOLIDATED STATEMENTS OF CASH FLOW',
                          'Statements of Consolidated Cash Flows']

        stock_related_texts = [
            'Stock-based compensation expense',
            'Stock-based compensation expense',
            'Share-based compensation expense',
            'Stock-based compensation',
            'Share-based compensation'
        ]

        all_ten_k_document_lists = []


    #     list_of_example_actual_full_ten_k_document_soup_doc = []

        # Define the group size
        group_size = 2
        items=[x]

        # Iterate over groups of items
        for start_index in range(0, len(items), group_size):
            group_items = items[start_index:start_index + group_size]

            list_of_example_actual_full_ten_k_document_soup_doc = []


    #         # Iterate over items within the group
    #         for item in group_items:
    #             print(item)

    #         # Add a separator for each group
    #         print("Group separator")




            url_to_parsed_content = {}


            for actual_full_ten_k_document_link in group_items:
                try:
                    actual_full_ten_k_document_link_collection = actual_full_ten_k_document_link
                    print(actual_full_ten_k_document_link)

                    for _ in range(3):
                        try:
                            actual_full_ten_k_document_content = await get_actual_full_ten_k_document_content(actual_full_ten_k_document_link)
                            example_actual_full_ten_k_document_soup_doc = await process_actual_full_ten_k_document(actual_full_ten_k_document_content)
                            list_of_example_actual_full_ten_k_document_soup_doc.append(example_actual_full_ten_k_document_soup_doc)
                                # Store the parsed content in the dictionary.
                            url_to_parsed_content[actual_full_ten_k_document_link] = example_actual_full_ten_k_document_soup_doc

                            break  # Break the loop if successful
                        except Exception as e:
                            print(f"An error occurred: {str(e)}. Retrying...")
                            time.sleep(15)
                except:
                    pass

            for actual_full_ten_k_document_link, one_actual_full_ten_k_document_soup_doc in url_to_parsed_content.items():
                ten_k_detail_list=[]

                if one_actual_full_ten_k_document_soup_doc:

                    elements = one_actual_full_ten_k_document_soup_doc.find_all(text=possible_titles)

                    if not elements:
                        print("Titles not found in the HTML.")
                        continue

                    for element in elements:
                        if element.strip() in possible_titles:
                            following_elements = element.find_all_next()
                            table = None

                            # Iterate through following elements and stop when a table is found or after three components
                            component_count = 0
                            for next_element in following_elements:
                                if next_element.name == 'table' and any(text in next_element.get_text() for text in stock_related_texts):
                                    table = next_element
                                    break
                                elif next_element.name in ['p', 'span']:
                                    component_count += 1
                                    if component_count >= 5:
                                        break

                            if table is None:
                                tables = one_actual_full_ten_k_document_soup_doc.find_all("table")
                                for t in tables:     
                                    if t:
                                        table_text = t.get_text()
                                        if any(title in table_text for title in possible_titles) and any(stock_related_text in table_text for stock_related_text in stock_related_texts):
                                            rows = t.find_all('tr')
                                            table_length = len(rows)
                                            print(f"Table Length from within: {len(rows)}") 
                                        break

                            if table:
                                rows = table.find_all('tr')
                                table_length=len(rows)

                                print(f"Table Length: {len(rows)}")                        

                                break



                    else:
                        print("No table found within or after the possible titles in the HTML.")

                else:
                    print("Document not processed, skipping")


                if table:
                    target_td = table.find('td', text=stock_related_texts)
                else:
                    target_td = None

                if target_td:
                    # Move up to the <tr> containing the <td> with "share-based compensation"
                    target_tr = target_td.find_parent('tr')

                    # Extract all the <td> values from the <tr> and put them in a column
                    values_in_column = [td.get_text(strip=True) for td in target_tr.find_all('td')]

                    # Output the values in the column
                    if values_in_column:
                        stock_stock_based_compensation_title=values_in_column[0]
                        list_of_stock_based_values=values_in_column
                        numbers_only_values_in_column = [entry for entry in values_in_column if any(char.isdigit() for char in entry)]
                        first_stock_based_compensation=numbers_only_values_in_column[0]
                        second_stock_based_compensation=numbers_only_values_in_column[1]
                        third_stock_based_compensation=numbers_only_values_in_column[2]
                        for value in values_in_column:
                            print(value)

                    ten_k_detail_list.append(actual_full_ten_k_document_link)
                    ten_k_detail_list.append(table_length)
                    ten_k_detail_list.append(list_of_stock_based_values)
                    ten_k_detail_list.append(first_stock_based_compensation)
                    ten_k_detail_list.append(second_stock_based_compensation)
                    ten_k_detail_list.append(third_stock_based_compensation)

                    first_rep = dict(zip(dict_titles, ten_k_detail_list))
                    all_ten_k_document_lists.append(first_rep)



                else:
                    print("The 'Share-Based Compensation' row not found!")
                time.sleep(10)

    #     print(all_ten_k_document_lists)
        return all_ten_k_document_lists


    # Run the main coroutine
    nest_asyncio.apply()

    # Use asyncio.run() to run the main coroutine
    all_ten_k_document_lists_of_dicts=asyncio.run(main())
    
    

In [ ]:
await extracting_stock_compensation_for_one_company("https://www.sec.gov//ix?doc=/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm")

In [ ]:
# this is final
async def extracting_stock_compensation_for_one_company(one_title_searchable):

    async def getting_10_k_link(one_title_searchable):
        #### ON THE SEC HOME PAGE
        # going to sec page
        sec_home_playwright = await async_playwright().start()
        sec_home_browser = await sec_home_playwright.chromium.launch(headless = False)
        sec_home_page = await sec_home_browser.new_page()

        await sec_home_page.goto("https://www.sec.gov/edgar/searchedgar/companysearch")
        time.sleep(6)

        try:

            # input company name (here is where we can enter title variable from df from csv)

            await sec_home_page.fill("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/input", one_title_searchable)
            time.sleep(6)

            # click search

            await sec_home_page.click("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/button")
            time.sleep(6)

            #============
            # ON THE NEW PAGE RESULTING WITH ALL COMPANY DOCS
            #input 10K for search
            await sec_home_page.fill("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[1]/input", "10-K")
            time.sleep(6)

            # click search

            await sec_home_page.click("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[5]/input[1]")
            time.sleep(6)


            #get all 10K entries
            list_of_all_ten_k_entries_response= await sec_home_page.content()

            list_of_all_ten_k_entries_doc = BeautifulSoup(list_of_all_ten_k_entries_response, 'html.parser')

            #get the first 10K
            list_of_all_ten_k_entries_table = list_of_all_ten_k_entries_doc.find('table', class_='tableFile2')

            ten_k_entries_rows = list_of_all_ten_k_entries_table.find_all('tr')  # get the first row because it has the most recent 10K

            most_recent_ten_k_link = None

            for one_ten_k_entries_row in ten_k_entries_rows:
                one_ten_k_entries_row_cells = one_ten_k_entries_row.find_all('td')  # Assuming each cell is represented by a <td> tag
            #     print(one_ten_k_entries_row_cells)

                for one_ten_k_entries_row_cell in one_ten_k_entries_row_cells:
            #         print(one_ten_k_entries_row_cell)
                    if one_ten_k_entries_row_cell.text.strip() == '10-K':
                        ten_k_document_link_href = one_ten_k_entries_row_cell.find_next('td').find('a')['href']

                        ten_k_document_link_base = "https://www.sec.gov/"
                        ten_k_document_link_tail = ten_k_document_link_href
                        ten_k_document_link = ten_k_document_link_base + ten_k_document_link_tail

            #             print(document_link)

                        # Store the first 10-K link and break out of the loop
                        most_recent_ten_k_link = ten_k_document_link
                        break

                if most_recent_ten_k_link is not None:
                    break


            # ONCE I HAVE THE TEN K DOC LINK 
            # 3. Now on the page with the document and graphics table

            ten_k_doc_link_playwright = await async_playwright().start()
            ten_k_doc_link_browser = await ten_k_doc_link_playwright.chromium.launch(headless = False)
            ten_k_doc_link_page = await ten_k_doc_link_browser.new_page()

            time.sleep(6)

            await ten_k_doc_link_page.goto(most_recent_ten_k_link)
            time.sleep(6)



            # Initiate the html and beautiful soup content 

            list_of_doc_and_graphics_response= await ten_k_doc_link_page.content()

            list_of_doc_and_graphics_soup_doc = BeautifulSoup(list_of_doc_and_graphics_response, 'html.parser')

            # get actual_full_ten_k_document_link
            list_of_doc_and_graphics_table = list_of_doc_and_graphics_soup_doc.find('table', class_='tableFile')


            list_of_doc_and_graphics_rows = list_of_doc_and_graphics_table.find_all('tr')  # Assuming each row is represented by a <tr> tag

            for list_of_doc_and_graphics_row in list_of_doc_and_graphics_rows:
                list_of_doc_and_graphics_cells = list_of_doc_and_graphics_row.find_all('td')  # Assuming each cell is represented by a <td> tag
            #     print(cells)
                for list_of_doc_and_graphics_cell in list_of_doc_and_graphics_cells:
                    if list_of_doc_and_graphics_cell.text.strip() == '10-K':
                        actual_document_link = list_of_doc_and_graphics_cell.find_next('td').find('a')
                        if actual_document_link is not None:
                            actual_ten_k_document_href = actual_document_link['href']
            #                 print("Documents href:", document_href)
                            actual_ten_k_document_link_base="https://www.sec.gov/"
                            actual_ten_k_document_link_tail=actual_ten_k_document_href
                            actual_full_ten_k_document_link=actual_ten_k_document_link_base+actual_ten_k_document_link_tail
                            x=actual_full_ten_k_document_link
                            print(actual_full_ten_k_document_link)
                            print("#########")


        except:
            pass

        finally:
            # Close the page and browser
            await ten_k_doc_link_page.close()
            await ten_k_doc_link_browser.close()
            await ten_k_doc_link_playwright.stop()

            # Close the sec_home page and browser
            await sec_home_page.close()
            await sec_home_browser.close()
            await sec_home_playwright.stop()
        return x

    # with change for link


    list_of_actual_full_ten_k_document_links = ["https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000156459022026876/msft-10k_20220630.htm"]

    actual_list_of_actual_full_ten_k_document_links = ["https://www.sec.gov//ix?doc=/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm","https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000156459022026876/msft-10k_20220630.htm","https://www.sec.gov/ix?doc=/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm"]

    async def get_actual_full_ten_k_document_content(x):
        playwright = await async_playwright().start()
        browser = await playwright.chromium.launch(headless=False)
    #     browser = await playwright.firefox.launch(headless=False)

        page = await browser.new_page()

        await page.goto(x)
        time.sleep(10)

        while True:
            # Get the number of spans on the page
            spans_assigned = await page.query_selector_all("span")
            test = len(spans_assigned)

            # Scroll in increments of 5000 pixels
            await page.evaluate('''() => {
                let distance = 5000;
                window.scrollBy(0, distance);
            }''')
    #         await asyncio.sleep(3)

            time.sleep(10)


            spans_assigned_two = await page.query_selector_all("span")
            test2 = len(spans_assigned_two)
            if test == test2:
                break

        # Get the content of the page
        actual_full_ten_k_document_content = await page.content()
        time.sleep(10)

        await browser.close()
        time.sleep(10)
        await page.close()  # Add this line to close the page
        time.sleep(10)

        return actual_full_ten_k_document_content


    async def process_actual_full_ten_k_document(actual_full_ten_k_document_content):
        actual_full_ten_k_document_soup = BeautifulSoup(actual_full_ten_k_document_content, 'html.parser')
        return actual_full_ten_k_document_soup


    async def main():
        dict_titles=['actual_full_ten_k_document_link_collection',
                     'table_length',
                     'list_of_stock_based_values',
                     'first_stock_based_compensation',
                     'second_stock_based_compensation',
                     'third_stock_based_compensation']

        possible_titles = ['CONSOLIDATED STATEMENTS OF CASH FLOWS',
                           'CASH FLOWS STATEMENTS',
                           'Consolidated Statements of Cash Flows',
                           'Consolidated Statement of Cash Flows',
                          'CONSOLIDATED STATEMENT OF CASH FLOWS',
                          'STATEMENTS OF CONSOLIDATED CASH FLOWS',
                          'Consolidated Cash Flow Statement',
                          'Consolidated Statement of Cash Flow for the Years Ended December 31',
                          'STATEMENT OF CASH FLOWS',
                          'CONSOLIDATED STATEMENTS OF CASH FLOW',
                          'Statements of Consolidated Cash Flows']

        stock_related_texts = [
            'Stock-based compensation expense',
            'Stock-based compensation expense',
            'Share-based compensation expense',
            'Stock-based compensation',
            'Share-based compensation'
        ]

        all_ten_k_document_lists = []


    #     list_of_example_actual_full_ten_k_document_soup_doc = []

        # Define the group size
        group_size = 2
        items=[x]

        # Iterate over groups of items
        for start_index in range(0, len(items), group_size):
            group_items = items[start_index:start_index + group_size]

            list_of_example_actual_full_ten_k_document_soup_doc = []


    #         # Iterate over items within the group
    #         for item in group_items:
    #             print(item)

    #         # Add a separator for each group
    #         print("Group separator")




            url_to_parsed_content = {}


            for actual_full_ten_k_document_link in group_items:
                try:
                    actual_full_ten_k_document_link_collection = actual_full_ten_k_document_link
                    print(actual_full_ten_k_document_link)

                    for _ in range(3):
                        try:
                            actual_full_ten_k_document_content = await get_actual_full_ten_k_document_content(actual_full_ten_k_document_link)
                            example_actual_full_ten_k_document_soup_doc = await process_actual_full_ten_k_document(actual_full_ten_k_document_content)
                            list_of_example_actual_full_ten_k_document_soup_doc.append(example_actual_full_ten_k_document_soup_doc)
                                # Store the parsed content in the dictionary.
                            url_to_parsed_content[actual_full_ten_k_document_link] = example_actual_full_ten_k_document_soup_doc

                            break  # Break the loop if successful
                        except Exception as e:
                            print(f"An error occurred: {str(e)}. Retrying...")
                            time.sleep(15)
                except:
                    pass

            for actual_full_ten_k_document_link, one_actual_full_ten_k_document_soup_doc in url_to_parsed_content.items():
                ten_k_detail_list=[]

                elements = one_actual_full_ten_k_document_soup_doc.find_all(text=possible_titles)

                if not elements:
                    print("Titles not found in the HTML.")
                    continue

                for element in elements:
                    if element.strip() in possible_titles:
                        following_elements = element.find_all_next()
                        table = None

                        # Iterate through following elements and stop when a table is found or after three components
                        component_count = 0
                        for next_element in following_elements:
                            if next_element.name == 'table' and any(text in next_element.get_text() for text in stock_related_texts):
                                table = next_element
                                break
                            elif next_element.name in ['p', 'span']:
                                component_count += 1
                                if component_count >= 5:
                                    break

                        if table:
                            rows = table.find_all('tr')
                            table_length=len(rows)

                            print(f"Table Length: {len(rows)}")                        

                            break

                else:
                    print("No table found after the possible titles in the HTML.")

                if table:
                    target_td = table.find('td', text=stock_related_texts)
                else:
                    target_td = None

                if target_td:
                    # Move up to the <tr> containing the <td> with "share-based compensation"
                    target_tr = target_td.find_parent('tr')

                    # Extract all the <td> values from the <tr> and put them in a column
                    values_in_column = [td.get_text(strip=True) for td in target_tr.find_all('td')]

                    # Output the values in the column
                    if values_in_column:
                        stock_stock_based_compensation_title=values_in_column[0]
                        list_of_stock_based_values=values_in_column
                        numbers_only_values_in_column = [entry for entry in values_in_column if any(char.isdigit() for char in entry)]
                        first_stock_based_compensation=numbers_only_values_in_column[0]
                        second_stock_based_compensation=numbers_only_values_in_column[1]
                        third_stock_based_compensation=numbers_only_values_in_column[2]
                        for value in values_in_column:
                            print(value)

                    ten_k_detail_list.append(actual_full_ten_k_document_link)
                    ten_k_detail_list.append(table_length)
                    ten_k_detail_list.append(list_of_stock_based_values)
                    ten_k_detail_list.append(first_stock_based_compensation)
                    ten_k_detail_list.append(second_stock_based_compensation)
                    ten_k_detail_list.append(third_stock_based_compensation)

                    first_rep = dict(zip(dict_titles, ten_k_detail_list))
                    all_ten_k_document_lists.append(first_rep)



                else:
                    print("The 'Share-Based Compensation' row not found!")
                time.sleep(10)

    #     print(all_ten_k_document_lists)
        return all_ten_k_document_lists


    # Run the main coroutine
    nest_asyncio.apply()

    # Use asyncio.run() to run the main coroutine
    all_ten_k_document_lists_of_dicts=asyncio.run(main())

In [ ]:
await extracting_stock_compensation_for_one_company("https://www.sec.gov//ix?doc=/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm")

In [ ]:
# with change for link
x=actual_full_ten_k_document_link

list_of_actual_full_ten_k_document_links = ["https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000156459022026876/msft-10k_20220630.htm"]

actual_list_of_actual_full_ten_k_document_links = ["https://www.sec.gov//ix?doc=/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm","https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000156459022026876/msft-10k_20220630.htm","https://www.sec.gov/ix?doc=/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm"]

async def get_actual_full_ten_k_document_content(x):
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless=False)
#     browser = await playwright.firefox.launch(headless=False)

    page = await browser.new_page()

    await page.goto(x)
    time.sleep(10)

    while True:
        # Get the number of spans on the page
        spans_assigned = await page.query_selector_all("span")
        test = len(spans_assigned)

        # Scroll in increments of 5000 pixels
        await page.evaluate('''() => {
            let distance = 5000;
            window.scrollBy(0, distance);
        }''')
#         await asyncio.sleep(3)

        time.sleep(10)


        spans_assigned_two = await page.query_selector_all("span")
        test2 = len(spans_assigned_two)
        if test == test2:
            break

    # Get the content of the page
    actual_full_ten_k_document_content = await page.content()
    time.sleep(10)

    await browser.close()
    time.sleep(10)
    await page.close()  # Add this line to close the page
    time.sleep(10)
    
    return actual_full_ten_k_document_content


async def process_actual_full_ten_k_document(actual_full_ten_k_document_content):
    actual_full_ten_k_document_soup = BeautifulSoup(actual_full_ten_k_document_content, 'html.parser')
    return actual_full_ten_k_document_soup


async def main():
    dict_titles=['actual_full_ten_k_document_link_collection',
                 'table_length',
                 'list_of_stock_based_values',
                 'first_stock_based_compensation',
                 'second_stock_based_compensation',
                 'third_stock_based_compensation']
    
    possible_titles = ['CONSOLIDATED STATEMENTS OF CASH FLOWS',
                       'CASH FLOWS STATEMENTS',
                       'Consolidated Statements of Cash Flows',
                       'Consolidated Statement of Cash Flows',
                      'CONSOLIDATED STATEMENT OF CASH FLOWS',
                      'STATEMENTS OF CONSOLIDATED CASH FLOWS',
                      'Consolidated Cash Flow Statement',
                      'Consolidated Statement of Cash Flow for the Years Ended December 31',
                      'STATEMENT OF CASH FLOWS',
                      'CONSOLIDATED STATEMENTS OF CASH FLOW',
                      'Statements of Consolidated Cash Flows']

    stock_related_texts = [
        'Stock-based compensation expense',
        'Stock-based compensation expense',
        'Share-based compensation expense',
        'Stock-based compensation',
        'Share-based compensation'
    ]

    all_ten_k_document_lists = []


#     list_of_example_actual_full_ten_k_document_soup_doc = []

    # Define the group size
    group_size = 1
    items=[x]
    
    # Iterate over groups of items
    for start_index in range(0, len(items), group_size):
        group_items = items[start_index:start_index + group_size]
        
        list_of_example_actual_full_ten_k_document_soup_doc = []


#         # Iterate over items within the group
#         for item in group_items:
#             print(item)

#         # Add a separator for each group
#         print("Group separator")




        url_to_parsed_content = {}


        for actual_full_ten_k_document_link in group_items:
            try:
                actual_full_ten_k_document_link_collection = actual_full_ten_k_document_link
                print(actual_full_ten_k_document_link)

                for _ in range(3):
                    try:
                        actual_full_ten_k_document_content = await get_actual_full_ten_k_document_content(actual_full_ten_k_document_link)
                        example_actual_full_ten_k_document_soup_doc = await process_actual_full_ten_k_document(actual_full_ten_k_document_content)
                        list_of_example_actual_full_ten_k_document_soup_doc.append(example_actual_full_ten_k_document_soup_doc)
                            # Store the parsed content in the dictionary.
                        url_to_parsed_content[actual_full_ten_k_document_link] = example_actual_full_ten_k_document_soup_doc

                        break  # Break the loop if successful
                    except Exception as e:
                        print(f"An error occurred: {str(e)}. Retrying...")
                        time.sleep(15)
            except:
                pass

        for actual_full_ten_k_document_link, one_actual_full_ten_k_document_soup_doc in url_to_parsed_content.items():
            ten_k_detail_list=[]

            elements = one_actual_full_ten_k_document_soup_doc.find_all(text=possible_titles)

            if not elements:
                print("Titles not found in the HTML.")
                continue

            for element in elements:
                if element.strip() in possible_titles:
                    following_elements = element.find_all_next()
                    table = None

                    # Iterate through following elements and stop when a table is found or after three components
                    component_count = 0
                    for next_element in following_elements:
                        if next_element.name == 'table' and any(text in next_element.get_text() for text in stock_related_texts):
                            table = next_element
                            break
                        elif next_element.name in ['p', 'span']:
                            component_count += 1
                            if component_count >= 5:
                                break

                    if table:
                        rows = table.find_all('tr')
                        table_length=len(rows)

                        print(f"Table Length: {len(rows)}")                        

                        break

            else:
                print("No table found after the possible titles in the HTML.")

            if table:
                target_td = table.find('td', text=stock_related_texts)
            else:
                target_td = None

            if target_td:
                # Move up to the <tr> containing the <td> with "share-based compensation"
                target_tr = target_td.find_parent('tr')

                # Extract all the <td> values from the <tr> and put them in a column
                values_in_column = [td.get_text(strip=True) for td in target_tr.find_all('td')]

                # Output the values in the column
                if values_in_column:
                    stock_stock_based_compensation_title=values_in_column[0]
                    list_of_stock_based_values=values_in_column
                    numbers_only_values_in_column = [entry for entry in values_in_column if any(char.isdigit() for char in entry)]
                    first_stock_based_compensation=numbers_only_values_in_column[0]
                    second_stock_based_compensation=numbers_only_values_in_column[1]
                    third_stock_based_compensation=numbers_only_values_in_column[2]
                    for value in values_in_column:
                        print(value)

                ten_k_detail_list.append(actual_full_ten_k_document_link)
                ten_k_detail_list.append(table_length)
                ten_k_detail_list.append(list_of_stock_based_values)
                ten_k_detail_list.append(first_stock_based_compensation)
                ten_k_detail_list.append(second_stock_based_compensation)
                ten_k_detail_list.append(third_stock_based_compensation)

                first_rep = dict(zip(dict_titles, ten_k_detail_list))
                all_ten_k_document_lists.append(first_rep)



            else:
                print("The 'Share-Based Compensation' row not found!")
            time.sleep(10)

#     print(all_ten_k_document_lists)
    return all_ten_k_document_lists


# Run the main coroutine
nest_asyncio.apply()

# Use asyncio.run() to run the main coroutine
all_ten_k_document_lists_of_dicts=asyncio.run(main())

In [ ]:
# this is final
async def extracting_stock_compensation_for_one_company(one_title_searchable):

    async def getting_10_k_link(one_title_searchable):
        #### ON THE SEC HOME PAGE
        # going to sec page
        sec_home_playwright = await async_playwright().start()
        sec_home_browser = await sec_home_playwright.chromium.launch(headless = False)
        sec_home_page = await sec_home_browser.new_page()

        await sec_home_page.goto("https://www.sec.gov/edgar/searchedgar/companysearch")
        time.sleep(6)

        try:

            # input company name (here is where we can enter title variable from df from csv)

            await sec_home_page.fill("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/input", one_title_searchable)
            time.sleep(6)

            # click search

            await sec_home_page.click("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/button")
            time.sleep(6)

            #============
            # ON THE NEW PAGE RESULTING WITH ALL COMPANY DOCS
            #input 10K for search
            await sec_home_page.fill("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[1]/input", "10-K")
            time.sleep(6)

            # click search

            await sec_home_page.click("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[5]/input[1]")
            time.sleep(6)


            #get all 10K entries
            list_of_all_ten_k_entries_response= await sec_home_page.content()

            list_of_all_ten_k_entries_doc = BeautifulSoup(list_of_all_ten_k_entries_response, 'html.parser')

            #get the first 10K
            list_of_all_ten_k_entries_table = list_of_all_ten_k_entries_doc.find('table', class_='tableFile2')

            ten_k_entries_rows = list_of_all_ten_k_entries_table.find_all('tr')  # get the first row because it has the most recent 10K

            most_recent_ten_k_link = None

            for one_ten_k_entries_row in ten_k_entries_rows:
                one_ten_k_entries_row_cells = one_ten_k_entries_row.find_all('td')  # Assuming each cell is represented by a <td> tag
            #     print(one_ten_k_entries_row_cells)

                for one_ten_k_entries_row_cell in one_ten_k_entries_row_cells:
            #         print(one_ten_k_entries_row_cell)
                    if one_ten_k_entries_row_cell.text.strip() == '10-K':
                        ten_k_document_link_href = one_ten_k_entries_row_cell.find_next('td').find('a')['href']

                        ten_k_document_link_base = "https://www.sec.gov/"
                        ten_k_document_link_tail = ten_k_document_link_href
                        ten_k_document_link = ten_k_document_link_base + ten_k_document_link_tail

            #             print(document_link)

                        # Store the first 10-K link and break out of the loop
                        most_recent_ten_k_link = ten_k_document_link
                        break

                if most_recent_ten_k_link is not None:
                    break


            # ONCE I HAVE THE TEN K DOC LINK 
            # 3. Now on the page with the document and graphics table

            ten_k_doc_link_playwright = await async_playwright().start()
            ten_k_doc_link_browser = await ten_k_doc_link_playwright.chromium.launch(headless = False)
            ten_k_doc_link_page = await ten_k_doc_link_browser.new_page()

            time.sleep(6)

            await ten_k_doc_link_page.goto(most_recent_ten_k_link)
            time.sleep(6)



            # Initiate the html and beautiful soup content 

            list_of_doc_and_graphics_response= await ten_k_doc_link_page.content()

            list_of_doc_and_graphics_soup_doc = BeautifulSoup(list_of_doc_and_graphics_response, 'html.parser')

            # get actual_full_ten_k_document_link
            list_of_doc_and_graphics_table = list_of_doc_and_graphics_soup_doc.find('table', class_='tableFile')


            list_of_doc_and_graphics_rows = list_of_doc_and_graphics_table.find_all('tr')  # Assuming each row is represented by a <tr> tag

            for list_of_doc_and_graphics_row in list_of_doc_and_graphics_rows:
                list_of_doc_and_graphics_cells = list_of_doc_and_graphics_row.find_all('td')  # Assuming each cell is represented by a <td> tag
            #     print(cells)
                for list_of_doc_and_graphics_cell in list_of_doc_and_graphics_cells:
                    if list_of_doc_and_graphics_cell.text.strip() == '10-K':
                        actual_document_link = list_of_doc_and_graphics_cell.find_next('td').find('a')
                        if actual_document_link is not None:
                            actual_ten_k_document_href = actual_document_link['href']
            #                 print("Documents href:", document_href)
                            actual_ten_k_document_link_base="https://www.sec.gov/"
                            actual_ten_k_document_link_tail=actual_ten_k_document_href
                            actual_full_ten_k_document_link=actual_ten_k_document_link_base+actual_ten_k_document_link_tail
                            actual_full_ten_k_document_link=x
                            print(actual_full_ten_k_document_link)
                            print("#########")

        except:
            pass

        finally:
            # Close the page and browser
            await ten_k_doc_link_page.close()
            await ten_k_doc_link_browser.close()
            await ten_k_doc_link_playwright.stop()

            # Close the sec_home page and browser
            await sec_home_page.close()
            await sec_home_browser.close()
            await sec_home_playwright.stop()

    # with change for link


    list_of_actual_full_ten_k_document_links = ["https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000156459022026876/msft-10k_20220630.htm"]

    actual_list_of_actual_full_ten_k_document_links = ["https://www.sec.gov//ix?doc=/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm","https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000156459022026876/msft-10k_20220630.htm","https://www.sec.gov/ix?doc=/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm"]

    async def get_actual_full_ten_k_document_content(x):
        playwright = await async_playwright().start()
        browser = await playwright.chromium.launch(headless=False)
    #     browser = await playwright.firefox.launch(headless=False)

        page = await browser.new_page()

        await page.goto(x)
        time.sleep(10)

        while True:
            # Get the number of spans on the page
            spans_assigned = await page.query_selector_all("span")
            test = len(spans_assigned)

            # Scroll in increments of 5000 pixels
            await page.evaluate('''() => {
                let distance = 5000;
                window.scrollBy(0, distance);
            }''')
    #         await asyncio.sleep(3)

            time.sleep(10)


            spans_assigned_two = await page.query_selector_all("span")
            test2 = len(spans_assigned_two)
            if test == test2:
                break

        # Get the content of the page
        actual_full_ten_k_document_content = await page.content()
        time.sleep(10)

        await browser.close()
        time.sleep(10)
        await page.close()  # Add this line to close the page
        time.sleep(10)

        return actual_full_ten_k_document_content


    async def process_actual_full_ten_k_document(actual_full_ten_k_document_content):
        actual_full_ten_k_document_soup = BeautifulSoup(actual_full_ten_k_document_content, 'html.parser')
        return actual_full_ten_k_document_soup


    async def main():
        dict_titles=['actual_full_ten_k_document_link_collection',
                     'table_length',
                     'list_of_stock_based_values',
                     'first_stock_based_compensation',
                     'second_stock_based_compensation',
                     'third_stock_based_compensation']

        possible_titles = ['CONSOLIDATED STATEMENTS OF CASH FLOWS',
                           'CASH FLOWS STATEMENTS',
                           'Consolidated Statements of Cash Flows',
                           'Consolidated Statement of Cash Flows',
                          'CONSOLIDATED STATEMENT OF CASH FLOWS',
                          'STATEMENTS OF CONSOLIDATED CASH FLOWS',
                          'Consolidated Cash Flow Statement',
                          'Consolidated Statement of Cash Flow for the Years Ended December 31',
                          'STATEMENT OF CASH FLOWS',
                          'CONSOLIDATED STATEMENTS OF CASH FLOW',
                          'Statements of Consolidated Cash Flows']

        stock_related_texts = [
            'Stock-based compensation expense',
            'Stock-based compensation expense',
            'Share-based compensation expense',
            'Stock-based compensation',
            'Share-based compensation'
        ]

        all_ten_k_document_lists = []


    #     list_of_example_actual_full_ten_k_document_soup_doc = []

        # Define the group size
        group_size = 2
        items=df_stock_top_us_plus_sec_titles['actual_full_ten_k_document_link']

        # Iterate over groups of items
        for start_index in range(0, len(items), group_size):
            group_items = items[start_index:start_index + group_size]

            list_of_example_actual_full_ten_k_document_soup_doc = []


    #         # Iterate over items within the group
    #         for item in group_items:
    #             print(item)

    #         # Add a separator for each group
    #         print("Group separator")




            url_to_parsed_content = {}


            for actual_full_ten_k_document_link in group_items:
                try:
                    actual_full_ten_k_document_link_collection = actual_full_ten_k_document_link
                    print(actual_full_ten_k_document_link)

                    for _ in range(3):
                        try:
                            actual_full_ten_k_document_content = await get_actual_full_ten_k_document_content(actual_full_ten_k_document_link)
                            example_actual_full_ten_k_document_soup_doc = await process_actual_full_ten_k_document(actual_full_ten_k_document_content)
                            list_of_example_actual_full_ten_k_document_soup_doc.append(example_actual_full_ten_k_document_soup_doc)
                                # Store the parsed content in the dictionary.
                            url_to_parsed_content[actual_full_ten_k_document_link] = example_actual_full_ten_k_document_soup_doc

                            break  # Break the loop if successful
                        except Exception as e:
                            print(f"An error occurred: {str(e)}. Retrying...")
                            time.sleep(15)
                except:
                    pass

            for actual_full_ten_k_document_link, one_actual_full_ten_k_document_soup_doc in url_to_parsed_content.items():
                ten_k_detail_list=[]

                if one_actual_full_ten_k_document_soup_doc:

                    elements = one_actual_full_ten_k_document_soup_doc.find_all(text=possible_titles)

                    if not elements:
                        print("Titles not found in the HTML.")
                        continue

                    for element in elements:
                        if element.strip() in possible_titles:
                            following_elements = element.find_all_next()
                            table = None

                            # Iterate through following elements and stop when a table is found or after three components
                            component_count = 0
                            for next_element in following_elements:
                                if next_element.name == 'table' and any(text in next_element.get_text() for text in stock_related_texts):
                                    table = next_element
                                    break
                                elif next_element.name in ['p', 'span']:
                                    component_count += 1
                                    if component_count >= 5:
                                        break

                            if table is None:
                                tables = one_actual_full_ten_k_document_soup_doc.find_all("table")
                                for t in tables:     
                                    if t:
                                        table_text = t.get_text()
                                        if any(title in table_text for title in possible_titles) and any(stock_related_text in table_text for stock_related_text in stock_related_texts):
                                            rows = t.find_all('tr')
                                            table_length = len(rows)
                                            print(f"Table Length from within: {len(rows)}") 
                                        break

                            if table:
                                rows = table.find_all('tr')
                                table_length=len(rows)

                                print(f"Table Length: {len(rows)}")                        

                                break



                    else:
                        print("No table found within or after the possible titles in the HTML.")

                else:
                    print("Document not processed, skipping")


                if table:
                    target_td = table.find('td', text=stock_related_texts)
                else:
                    target_td = None

                if target_td:
                    # Move up to the <tr> containing the <td> with "share-based compensation"
                    target_tr = target_td.find_parent('tr')

                    # Extract all the <td> values from the <tr> and put them in a column
                    values_in_column = [td.get_text(strip=True) for td in target_tr.find_all('td')]

                    # Output the values in the column
                    if values_in_column:
                        stock_stock_based_compensation_title=values_in_column[0]
                        list_of_stock_based_values=values_in_column
                        numbers_only_values_in_column = [entry for entry in values_in_column if any(char.isdigit() for char in entry)]
                        first_stock_based_compensation=numbers_only_values_in_column[0]
                        second_stock_based_compensation=numbers_only_values_in_column[1]
                        third_stock_based_compensation=numbers_only_values_in_column[2]
                        for value in values_in_column:
                            print(value)

                    ten_k_detail_list.append(actual_full_ten_k_document_link)
                    ten_k_detail_list.append(table_length)
                    ten_k_detail_list.append(list_of_stock_based_values)
                    ten_k_detail_list.append(first_stock_based_compensation)
                    ten_k_detail_list.append(second_stock_based_compensation)
                    ten_k_detail_list.append(third_stock_based_compensation)

                    first_rep = dict(zip(dict_titles, ten_k_detail_list))
                    all_ten_k_document_lists.append(first_rep)



                else:
                    print("The 'Share-Based Compensation' row not found!")
                time.sleep(10)

    #     print(all_ten_k_document_lists)
        return all_ten_k_document_lists


    # Run the main coroutine
    nest_asyncio.apply()

    # Use asyncio.run() to run the main coroutine
    all_ten_k_document_lists_of_dicts=asyncio.run(main())

In [ ]:
await extracting_stock_compensation_for_one_company(one_title_searchable)